# motor

> Calculations and other programs for a 1959 Triumph TR3A conversion to electric drive.

In [1]:
#| default_exp motor

In [2]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [3]:
#| export
import math
from triumph.utilities import *

### Determining the Tire Radius

The 1959 Triumph TR3A typically came equipped with 15-inch wheels. To calculate the wheel radius, you need to take into account both the wheel diameter and the tire profile. In this case, let's consider a tire size of 165/80R15, which is a common tire size used on this vehicle.

To find the tire radius, first calculate the sidewall height by multiplying the tire width (165 mm) by the aspect ratio (80%). Then convert this to inches:

Sidewall height (in inches) = (165 mm × 0.80) / 25.4 ≈ 5.20 inches

Next, add the sidewall height to half of the wheel diameter (7.5 inches):

Tire radius (in inches) = 7.5 inches (wheel radius) + 5.20 inches (sidewall height) ≈ 12.70 inches

So, the approximate tire radius for a 1959 Triumph TR3A with 165/80R15 tires is 12.7 inches. This value may vary depending on the specific tire model and inflation pressure. 

In [4]:
#| export
def tire_radius(tire:str="165/80R15" # tire size in the format of "165/80R15"
                )->float:  # tire radius in inches
    """Calculate the tire radius in inches
    """
    wheel_diameter_inches = int(tire[-2:]) 
    sidewall_height = int(tire[0:3]) # sidewall height in mm
    aspect_ratio = int(tire.split("/")[1][0:2]) # aspect ratio
    sidewall_height_inches = sidewall_height * (aspect_ratio / 100) / 25.4 # sidewall height in inches
    radius = wheel_diameter_inches / 2 + sidewall_height_inches
    return radius


# Convert Engine, Wheel and Gear information into a 0-to-60 time

To determine how many seconds it will take to accelerate to 60 mph, we need to know the mass of the car, the force acting on the car, and the acceleration of the car.

The acceleration of the car is given by Newton's second law of motion: $F = M * A$.

The force moving the car forward is governed by the engine torque, but modified by the gear ratio and the radius of the tires.

Specifically, engine torque ($T$, [Nm]) is multiplied by the gear ratio ($G$), which provides a total torque acting on the tires. In addition, the force acting on the tires is not the same as the torque between the tires and the pavement. The torque between the tires and the pavement is determined by the radius of the tires, $R$ [m]: 

$$F = T * G / R$$

### Torque to Force Conversion:

In [5]:
#| export
def Torque_to_Force(torque:float, # torque in Nm
                    gear_ratio:float, # gear ratio
                    radius: float # radius of the wheel in m
                    )->float:   # Force acting on the pavement to push the car (N)
    """ Convert engine torque to force acting on pavement to push the car """
    return torque * gear_ratio / radius

### Test the function

Assume we have a vehicle with a torque of 220 Nm and a gear ratio of 3.7. Further assume that the tires have a radius of 0.3 m.

In this case, the force acting on the tires is given by: 

$$ 220 \text{ Nm} * 3.7 / 0.3 \text{ m} = 2713.33 \text{ N}. $$

In [6]:
torque=220 # engine torque in Nm
gear_ratio=3.7 # gear ratio
radius=0.3 # radius of the wheel in m

F = Torque_to_Force(torque, gear_ratio, radius)
print(f"The force acting on the pavement to push the car is {round(F,2)} N")
test_eq(round(F,2),2713.33)

The force acting on the pavement to push the car is 2713.33 N


### Determining Acceleration
To determine how fast the vehicle will accelerate, we need to know the mass of the vehicle. Assume the vehicle has a mass of 2000 lbs.

In [7]:
#| export
def Acceleration(F:float, # Force in N
                 mass:float, # mass of the car in kg
                 )->float: # acceleration in m/s^2
    """ Calculate the acceleration of the car """
    return F / mass

def zero_to_sixty(A:float # acceleration in m/s^2
                  )->float: # time in seconds
    """ Calculate the time it takes to go from 0 to 60 mph """
    # convert A from m/s^2 to mph/s
    A = A * 2.23693629 
    return 60 / A


In [8]:
GVWR = 2000 # lbs
GVWR_kg = lbs_to_Kg(GVWR)
A = Acceleration(F, GVWR_kg)
print(f"Acceleration: {round(A,2)} m/s^2")
test_eq(round(A,2),2.99)

Acceleration: 2.99 m/s^2


In [9]:
zero_to_sixty(A)
print(
    f"It will take {round(zero_to_sixty(A),2)} seconds to go from 0 \
to 60 mph under ideal conditions."
)
test_eq(round(zero_to_sixty(A),2),8.97)

It will take 8.97 seconds to go from 0 to 60 mph under ideal conditions.


## Determining Top Speed

The top speed of the car will be determined by the motor RPMs, the gear ratio, and the radius of the tires.

In [10]:
#| export
def top_speed(rpm_max:float, # max rpm
              gear_ratio:float, # gear ratio
              radius:float # radius of the wheel in m
              )->float: # top speed in mph
    """ Calculate the top speed of the car """
    return rpm_max * 60 * meters_to_inches(radius) * math.pi / (gear_ratio * 63360)

In [11]:
top_speed(7000, 3.7, 0.3)

66.47675446147143

In [12]:
rpm_max = 8000 # rpm
gear_ratio = 3.7 # gear ratio
radius = 0.3 # radius of the wheel in m

mph_max = top_speed(rpm_max, gear_ratio, radius)
print(f"Top speed: {round(mph_max)} mph")

test_eq(round(mph_max), 76)

Top speed: 76 mph


### RPMs at a given speed.

Knowing the tire dimensions and gear ratio, we can determine the RPM of the engine for any given speed.

In [13]:
#| export
def rpm_at_mph(gear_ratio:float, # gear ratio
                 tire:str, # tire size in the format of "165/80R15"
                 mph:float=60 # speed in mph (default is 60 mph)
                 )->float: # rpm
    """ Calculate the rpm at a given speed """
    radius = tire_radius(tire) # tire radius in inches
    tire_circumference = 2 * math.pi * radius # tire circumference in inches
    tire_rpm = mph * 5280 * 12 / (tire_circumference * 60) # tire rpm 
    return round(tire_rpm * gear_ratio)

In [14]:
print("The RPM at 60 mph in second gear (2.0): ", rpm_at_mph(2.0, "165/80R15", 60))
print("The RPM at 100 mph in fourth gear (1.0): ", rpm_at_mph(1.0, "165/80R15", 100))
test_eq(rpm_at_mph(1, "165/80R15", 100),1324)
test_eq(rpm_at_mph(2.0, "165/80R15", 60),1588)

The RPM at 60 mph in second gear (2.0):  1588
The RPM at 100 mph in fourth gear (1.0):  1324


## Exploring the impacts of the gear ratio on acceleration and top speed

We want to determine an ideal gear ratio that maximizes the acceleration and top speed of the vehicle. To do this, we'll need to create a function that takes the gear ratio as an input and returns the acceleration and top speed.

In [15]:
#| export
def evaluate(gear_ratio:float, # gear ratio
             GVWR:float, # gross vehicle weight rating in kg
             tire:str, # tire size in the format of "165/80R15"
             rpm_max:int, # max rpm
             torque:float # torque in Nm
             ):
    """ Evaluate the car """
    radius = tire_radius(tire)/12 # tire radius in feet
    radius = feet_to_meters(radius)
    F = Torque_to_Force(torque, gear_ratio, radius)
    A = Acceleration(F, GVWR)
    to60 = zero_to_sixty(A)
    mph_max = top_speed(rpm_max, gear_ratio, radius)
    return {"zero to sixty (sec)":round(to60,1), "Top speed (mph)":round(mph_max)}

In [16]:
#| hide
import nbdev; nbdev.nbdev_export()